In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
import os
#import xgboost as xgb
#import lightgbm as lgb
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import gc
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [13]:
train = pd.read_csv('dataset-0510/train.csv')
test  = pd.read_csv('dataset-0510/test.csv')


In [14]:
data = pd.concat([train, test], axis=0)
gc.collect()

834

In [100]:
def feature_process(df):
    
    df['min_cat'] = 0
    for col in [i for i in df.columns if 'MIN' in i]:
        df['min_cat'] = df.apply(lambda x : col if x['min_cat'] == x[col] else x['min_cat'], axis=1)  
    
    #Impute missing value
    df['village_income_mean'] = df.groupby(['city', 'town'])['village_income_median'].transform(lambda x : x .fillna(x.mean()))
    
    #location

    df['location_2'] = df.apply(lambda x : int(str(x['city']) + str(x['town'])), axis=1)
    df['city'] = df['city'].astype('category')
    df['location_2'] = df['location_2'].astype('category')
    
    #degree rate
    df['diff_doc_master'] = df['doc_rate'] - df['master_rate']
    df['diff_master_bachelor'] = df['master_rate'] - df['bachelor_rate']
    df['diff_bachelor_highsch'] = df['bachelor_rate'] - df['highschool_rate']
    df['diff_highsch_jobschool'] = df['highschool_rate'] - df['jobschool_rate']
    df['diff_jobschool_elesch'] = df['jobschool_rate'] - df['elementary_rate']
    
    df['all_degree'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] + df['jobschool_rate'] + df['junior_rate'] + df['elementary_rate']
    df['junior_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] + df['jobschool_rate'] + df['junior_rate'] 
    df['jobschool_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] +  df['jobschool_rate']
    df['highschool_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate']
    df['bachelor_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate']
    df['master_above_rate'] = df['doc_rate'] + df['master_rate'] 
    
    #building
    df['building_use'] = df['building_use'].astype('category')
    df['building_material'] = df['building_material'].astype('category')
    
    #floor
    df.loc[df['txn_floor'].isna(), 'department'] = 1 
    df.loc[df['txn_floor'].notna(), 'department'] = 0
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor'] 
    df['avg_height_floor'].fillna(0, inplace = True)
    
    #area
    df['land/bulid_area'] = df['land_area'] / df ['building_area']
    df['parking_rate'] = df['building_area'] / df['parking_area']
    
    #date
    df['day_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['year_between_txn_complete'] = round(df['day_between_txn_complete'] / 365)
    
    #soical rate
    df['natural_diff'] = df['born_rate'] - df['death_rate']
    df['natural_rate'] = df['born_rate'] / df['death_rate']
    
    df['marry_diff'] = df['marriage_rate'] - df['divorce_rate']
    df['marry_rate'] = df['marriage_rate'] / df['divorce_rate']
    
    df['total_diff_sum'] = df['natural_rate'] + df['marry_diff']
    df['total_diff_diff'] = df['natural_rate'] - df['marry_diff']
    df['total_rate_diff'] = df['born_rate'] + df['marry_rate']
    
    df['positive_grow_rate'] = df['born_rate'] + df['marriage_rate']
    df['negative_grow_rate'] = df['death_rate'] + df['divorce_rate']
    df['tatal_rate_sum'] = df['born_rate'] + df['death_rate'] + df['born_rate'] + df['death_rate']
    
    #village_income_median
    temp = df.groupby(['city', 'town', 'village'])['village_income_median'].first().reset_index()
    temp = df.groupby(['city', 'town']).agg({'village_income_median' : ['mean', 'sum', 'median']})
    temp = temp.rename({'village_income_median' : 'town_income_median'}, level = 0)
    temp.columns = [e[0] + '_' + e[1] if e[1] != '' else e[0] for e in temp.columns.tolist()]
    df = pd.merge(df, temp, on = ['city', 'town'], how = 'left')

    temp = df.groupby(['city', 'town', 'village'])['village_income_median'].first().reset_index()
    temp = df.groupby(['city']).agg({'village_income_median' : ['mean', 'sum', 'median']})
    temp = temp.rename({'village_income_median' : 'city_income_median'}, level = 0)
    temp.columns = [e[0] + '_' + e[1] if e[1] != '' else e[0] for e in temp.columns.tolist()]
    df = pd.merge(df, temp, on = ['city'], how = 'left')
    
    #interection 
    '''
    locations =['city', 'location2']
    inter_cols = ['building_type', 'building_use', 'building_material']
    for location in locations:
        for inter_col in inter_cols:
            df['inter_btw_' + location +'_' + inter_col] = df.apply(lambda x : str(x[location]) + str(x[inter_col]), axis=1)
            df['inter_btw_' + location +'_' + inter_col] = df['inter_btw_' + location +'_' + inter_col].astype('category')
    '''
    inter_cols = ['building_type', 'building_use', 'building_material']
    for i in range(3):
        for j in range(3):
            if j > i:
                df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]] = df.apply(lambda x : str(x[inter_cols[i]]) + str(x[inter_cols[j]]), axis=1)
                df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]] = df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]].astype('category')
    #one-hot
    df = df.join(pd.get_dummies(df['parking_way'], prefix = 'parking_way'))
    df = df.join(pd.get_dummies(df['building_type'], prefix = 'building_type'))
    df = df.join(pd.get_dummies(df['building_material'], prefix = 'building_material'))
    df = df.join(pd.get_dummies(df['building_use'], prefix = 'building_use'))
    
    #useless cols
    df.drop(['village', 'town', 'building_type', 'parking_way','building_material', 'building_use', 'lat', 'lon'], axis = 1, inplace = True)
    df.drop([i for i in train.columns if np.sum(train[i]) == 60000 and 'index' in i], axis = 1, inplace = True)
    return df

In [101]:
final_data = feature_process(data)

In [102]:
final_data = feature_process(data)
FE_train = final_data[:60000]
FE_test = final_data[60000:]
FE_train.to_csv('FE_train.csv', index = False)
FE_test.to_csv('FE_test.csv', index = False)
len(FE_train.columns), len(FE_test.columns)

(271, 271)

In [103]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 235 entries, building_id to total_price
dtypes: float64(37), int64(197), object(1)
memory usage: 107.6+ MB


In [99]:
[i for i in FE_train.columns if FE_train[i].dtypes not in ['float', 'int', 'O']]

['building_material',
 'building_use',
 'location_2',
 'inter_btw_building_type_building_use',
 'inter_btw_building_type_building_material',
 'inter_btw_building_use_building_material',
 'parking_way_0',
 'parking_way_1',
 'parking_way_2',
 'building_type_0',
 'building_type_1',
 'building_type_2',
 'building_type_3',
 'building_type_4']

In [87]:
train.groupby(['city', 'town', 'village'])